
# Practicum II - Movie Database Applicaton
### Collaborators: Abasamia Akpan, Rebecca Brent and Alessia Pizzoccheri

This Notebook provides context, documentation and sample code for a Database Application using IMBd datasets; for clarity, this document has been divided into the following sections:

### 1. Introduction
### 2. Assumptions
### 3. Logical Models
### 4. Database and Relations Definition¶
### 1. Introduction
#### 1.1. Looking at the data
In this section, we analyze and deconstruct the various data sets found on the IMBd page; data is organized in seven TSV files, specifically:

Title AKAs
Title Basics
Title Episodes
Title Ratings
Title Crew
Name Basics
Title Principles Our first step is to establish relationships between each data set and identify a primary key (PK) on each table. Since no information was provided regarding the role of columns, we devised a solution to find unique identifiers within each data set; that is, for each table we compare the total number of rows against the total number of unique values for a given column.
1.2 Establishing PKs
Following the system aforementioned, we identified the following primary keys

In [1]:

import pandas as pd
import numpy as np
import os
import sys
import pip
pip.main(['install','prettytable'])
pip.main(['install','mysql-connector-python'])
pip.main(['install','tqdm'])
pip.main(['install','pymysql'])

from tqdm import tqdm



Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [12]:
import mysql.connector
from prettytable import PrettyTable 
from prettytable import from_csv
from mysql.connector import Error

def create_connection_nodb(host_name, user_name, user_password):
    """ create a connection to mysql host
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_database(connection, db_name):
    """ create a database 
    :param connection: Connection object
    :param db_name: name of database in form of string
    :return:
    """
    cursor = connection.cursor()
    try:
        query = "CREATE DATABASE IF NOT EXISTS " + db_name
        cursor.execute(query)
        print("Database created successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def create_connection(host_name, user_name, user_password, db_name):
    """ create a connection to the (possibly newly created) database
    :param host_name: string host name (ie. "localhost")
    :param user_name: string username (ie. "root")
    :param user_password: string the password to connect to mysql
    :param db_name: string name of database 
    :return: connection object
    """
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
        table_name = create_table_sql.split(" ")[5].split("(")[0]
        print("Successfully created table: " + table_name)
    except Error as e:
        print(e)
        
def main():
    
    database = "imbd_application"
    host_name = "localhost"
    user_name = "root"
    user_password = "Odyssey22"
    
    create_table_titlebasics = """CREATE TABLE IF NOT EXISTS titlebasics(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    titleType TEXT,
                                    primaryTitle TEXT,
                                    originalTitle TEXT,
                                    isAdult BOOLEAN,
                                    startYear DATE,
                                    endYear DATE,
                                    runTimeMinutes TIME)"""
    
    create_table_titleratings = """CREATE TABLE IF NOT EXISTS titleratings(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    averageRating FLOAT,
                                    numVotes INT,
                                    FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""


    create_table_titleepisodes = """CREATE TABLE IF NOT EXISTS titleepisodes(
                                    tconst VARCHAR(11) PRIMARY KEY,
                                    seasonNumber INT,
                                    episodeNumber INT,
                                    parentTconst CHAR(9),
                                    FOREIGN KEY (parentTconst) REFERENCES titlebasics(tconst))"""

    create_table_titleAKAs = """CREATE TABLE IF NOT EXISTS titleAKAs(
                                akasID INT AUTO_INCREMENT PRIMARY KEY,
                                titleID VARCHAR(11),
                                ordering INT,
                                title TEXT,
                                region TEXT,
                                language TEXT,
                                isoriginaltitle BOOLEAN,
                                FOREIGN KEY (titleID) REFERENCES titlebasics(tconst))"""

    create_table_namebasics = """CREATE TABLE IF NOT EXISTS namebasics(
                                nconst VARCHAR(11) PRIMARY KEY,
                                primaryname TEXT,
                                birthyear DATE,
                                deathyear DATE,
                                noofmovies INT,
                                age INT,
                                currentdate DATE)"""
    
    create_table_principals = """CREATE TABLE IF NOT EXISTS principals(
                                principalsID INT AUTO_INCREMENT PRIMARY KEY,
                                tconst VARCHAR(11),
                                ordering INT,
                                category TINYTEXT,
                                job TINYTEXT,
                                nconst VARCHAR(11),
                                FOREIGN KEY (tconst) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (nconst) REFERENCES namebasics(nconst))"""
    
    create_table_titlecrew = """CREATE TABLE IF NOT EXISTS titlecrew(
                                crewid INT AUTO_INCREMENT PRIMARY KEY,
                                tconst VARCHAR(11) UNIQUE,
                                FOREIGN KEY (tconst) REFERENCES titlebasics(tconst))"""
    
    create_table_characters = """CREATE TABLE IF NOT EXISTS characters(
                                characterID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(256) UNIQUE)"""
    
    create_table_writers = """CREATE TABLE IF NOT EXISTS writers(
                                writerID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(256) UNIQUE)"""
    
    create_table_directors = """CREATE TABLE IF NOT EXISTS directors(
                                directorID INT AUTO_INCREMENT PRIMARY KEY,
                                name VARCHAR(256) UNIQUE)"""
    
    create_table_genres = """CREATE TABLE IF NOT EXISTS genres(
                                genreID INT AUTO_INCREMENT  PRIMARY KEY,
                                genre VARCHAR(256) UNIQUE)"""
    
    create_table_knownfortitles = """CREATE TABLE IF NOT EXISTS knownfortitles(
                                knownfortitlesID INT AUTO_INCREMENT PRIMARY KEY,
                                titles VARCHAR(11) UNIQUE)"""
    
    create_table_primaryprofession = """CREATE TABLE IF NOT EXISTS primaryprofession(
                                primaryprofessionID INT AUTO_INCREMENT PRIMARY KEY,
                                profession VARCHAR(256) UNIQUE)"""
    
    create_table_types = """CREATE TABLE IF NOT EXISTS types(
                                typeID INT AUTO_INCREMENT  PRIMARY KEY,
                                type VARCHAR(256) UNIQUE)"""
    
    create_table_attributes = """CREATE TABLE IF NOT EXISTS attributes(
                                attributeID INT AUTO_INCREMENT PRIMARY KEY,
                                attribute VARCHAR(256) UNIQUE)"""
    
    create_table_genretitlebasics = """CREATE TABLE IF NOT EXISTS genretitlebasics(
                                titlebasicsID VARCHAR(11),
                                genreID INT,
                                PRIMARY KEY (titlebasicsID, genreID),
                                FOREIGN KEY (titlebasicsID) REFERENCES titlebasics(tconst),
                                FOREIGN KEY (genreID) REFERENCES genres(genreID))"""
    
    create_table_crewdirectors = """CREATE TABLE IF NOT EXISTS crewdirectors(
                                crewid INT,
                                directorid INT,
                                PRIMARY KEY (crewid ,directorid),
                                FOREIGN KEY (crewid) REFERENCES titlecrew(crewid),
                                FOREIGN KEY (directorid) REFERENCES directors(directorID))"""
    
    create_table_crewwriters = """CREATE TABLE IF NOT EXISTS crewwriters(
                                crewid INT,
                                writerid INT,
                                PRIMARY KEY (crewid, writerid),
                                FOREIGN KEY (crewid) REFERENCES titlecrew(crewid),
                                FOREIGN KEY (writerid) REFERENCES writers(writerid))"""
    
    create_table_principalscharacters = """CREATE TABLE IF NOT EXISTS principalscharacters(
                                principalsid INT,
                                characterid INT,
                                PRIMARY KEY (principalsid, characterid),
                                FOREIGN KEY (principalsid) REFERENCES principals(principalsID),
                                FOREIGN KEY (characterid) REFERENCES characters(characterid))"""
    
    create_table_AKAattributes = """CREATE TABLE IF NOT EXISTS AKAattributes(
                                akasid INT,
                                attributeid INT,
                                PRIMARY KEY (akasid, attributeid),
                                FOREIGN KEY (akasid) REFERENCES titleAKAs(AKAsID),
                                FOREIGN KEY (attributeid) REFERENCES attributes(attributeid))"""
    
    create_table_AKAtype = """CREATE TABLE IF NOT EXISTS AKAtype(
                                akasid INT,
                                typeid INT,
                                PRIMARY KEY (akasid, typeid),
                                FOREIGN KEY (akasid) REFERENCES titleAKAs(AKAsID),
                                FOREIGN KEY (typeid) REFERENCES types(typeid))"""
    
    create_table_primaryprofessiontitlenames = """CREATE TABLE IF NOT EXISTS primaryprofessiontitlenames(
                                primaryprofessionid INT,
                                namebasicsid VARCHAR(11),
                                PRIMARY KEY (primaryprofessionid, namebasicsid),
                                FOREIGN KEY (primaryprofessionid) REFERENCES primaryprofession(primaryprofessionid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst))"""
    
    create_table_knownfortitlenames = """CREATE TABLE IF NOT EXISTS knownfortitlenames(
                                namebasicsid VARCHAR(11),
                                knownfortitlesid INT,
                                PRIMARY KEY (namebasicsid, knownfortitlesid),
                                FOREIGN KEY (namebasicsid) REFERENCES namebasics(nconst),
                                FOREIGN KEY (knownfortitlesid) REFERENCES knownfortitles(knownfortitlesid))"""
    
    conn = create_connection_nodb(host_name, user_name, user_password)
    
    # create database object if not already created and then connect the cursor to it
    if conn is not None:
        create_database(conn, database)
        conn = create_connection(host_name, user_name, user_password, database)
        
        # create tables
        if conn is not None:
            
            # create Title Basics table
            create_table(conn, create_table_titlebasics)

            # create Title Ratings table
            create_table(conn, create_table_titleratings)
            
            # create title episodes table
            create_table(conn, create_table_titleepisodes)
            
            # create title AKAs table
            create_table(conn, create_table_titleAKAs)
            
            # create name basics table
            create_table(conn, create_table_namebasics)
            
            # create principals table
            create_table(conn, create_table_principals)
            
            # create title crew table
            create_table(conn, create_table_titlecrew)
            
            # create characters table
            create_table(conn,create_table_characters)
            
            # create writers table
            create_table(conn,create_table_writers)
            
            # create directors table
            create_table(conn,create_table_directors)
            
            # create genres table
            create_table(conn,create_table_genres)
            
            # create known for titles table
            create_table(conn,create_table_knownfortitles)
            
            # create primary profession table
            create_table(conn,create_table_primaryprofession)
            
            # create type table
            create_table(conn,create_table_types)
            
            # create attributes table
            create_table(conn,create_table_attributes)
            
            # create genre title basics table
            create_table(conn, create_table_genretitlebasics)
            
            # create crew directors table
            create_table(conn,create_table_crewdirectors)
            
            # create crew writers table
            create_table(conn,create_table_crewwriters)
            
            # create principals characters table
            create_table(conn, create_table_principalscharacters)
            
            # create AKA Attributes table
            create_table(conn, create_table_AKAattributes)
            
            # create AKA Type table
            create_table(conn, create_table_AKAtype)
            
            # create Primary Profession Title Names table
            create_table(conn, create_table_primaryprofessiontitlenames)
            
            # create known for title names table
            create_table(conn, create_table_knownfortitlenames)
                
        else:
            print("Error! cannot create the database connection.")
    
    else:
        print("Error! Cannot create the MySQL connection.")
    
    


if __name__ == '__main__':
    main()


Connection to MySQL DB successful
Database created successfully
Connection to MySQL DB successful


DatabaseError: 3730 (HY000): Cannot drop table 'titlebasics' referenced by a foreign key constraint 'titleratings_ibfk_1' on table 'titleratings'.

## SQL communication helper ##

In [3]:
pip.main(['install','sqlalchemy'])
import sqlalchemy
database_username = 'root'
database_password = 'Odyssey22'
database_ip       = 'localhost'
database_name     = 'imbd_application'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
You should consider upgrading via the 'c:\users\rbren\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [4]:
## Helper Functions

In [5]:
def load_dataframe(data_frame,table_name):
    data_frame_new=data_frame
    
    data_frame_new.to_sql(table_name,con=database_connection, if_exists='append',index=False)
    
    return 

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

## Title Basics

In [6]:
# read title basic file
title_basics = pd.read_csv('title.basics.tsv', delimiter='\t',encoding='utf-8')
title_basics

c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
7316917,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
7316918,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
7316919,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
7316920,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [7]:
def preprocess_dataframe_title_basics(data_frame):
    title_basics_new_series=data_frame

    b = pd.to_datetime(title_basics_new_series['runtimeMinutes'], errors='coerce', format='%M')

    b.to_frame().runtimeMinutes
    df_1 = pd.DataFrame(dict(time_stamps = b)) 

    df_1['runtimeMinutes'] = df_1['time_stamps'].dt.minute
    title_basics_new_series=title_basics_new_series.drop(['runtimeMinutes'],axis=1)
    title_basics_new_series['runtimeMinutes']=df_1['runtimeMinutes']

    #print(title_basics_new_series)
    
    title_basics_new_frame_transpose=title_basics_new_series.astype({'isAdult': 'int32'})
    
    title_basics_new_frame_transpose["isAdult"]=title_basics_new_frame_transpose.isAdult.mask(title_basics_new_frame_transpose.isAdult > 1,1)
    #g.isAdult=g.isAdult.mask(g.isAdult > 1,1)

    #needs to be fixed
    title_basics_new_frame_transpose['startYear']= pd.to_datetime(title_basics.startYear, errors='coerce', format='%Y')

    #title_basics_new_frame_transpose['startYear']=[time.date() for time in title_basics_new_frame_transpose['startYear']]

    
    title_basics_new_frame_transpose['endYear']= pd.to_datetime(title_basics.endYear, errors='coerce', format='%Y')
   # title_basics_new_frame_transpose['endYear']=[time.date() for time in title_basics_new_frame_transpose['endYear']]

    title_basics_new_frame_transpose=title_basics_new_frame_transpose.drop(['genres'],axis=1)
    #print(title_basics_new_frame_transpose)
    
    #title_basics_new_frame_transpose.to_sql('titlebasics',con=database_connection, if_exists='append',index=False)
    new_row = {
        'tconst':None, 
        'titleType':None, 
        'primaryTitle':None,
        'originalTitle':None,
        'isAdult':None, 
        'startYear':None,
        'endYear':None,
        'runtimeMinutes':None         
    } 
  
    title_basics_new_frame_transpose = title_basics_new_frame_transpose.append(new_row, ignore_index=True)
    return title_basics_new_frame_transpose

title_basics_preprocessed=preprocess_dataframe_title_basics(title_basics)

In [8]:
len(title_basics_preprocessed)
title_basics_preprocessed

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes
0,tt0000001,short,Carmencita,Carmencita,0,1894-01-01,NaT,1.0
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892-01-01,NaT,5.0
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892-01-01,NaT,4.0
3,tt0000004,short,Un bon bock,Un bon bock,0,1892-01-01,NaT,12.0
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893-01-01,NaT,1.0
...,...,...,...,...,...,...,...,...
7316918,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010-01-01,NaT,NaN
7316919,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010-01-01,NaT,NaN
7316920,tt9916856,short,The Wind,The Wind,0,2015-01-01,NaT,27.0
7316921,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014-01-01,NaT,10.0


In [9]:
title_basics_preprocessed_new=title_basics_preprocessed.rename(columns={"runtimeMinutes": "runTimeMinutes"})

title_basics_preprocessed_new.runTimeMinutes

0           1.0
1           5.0
2           4.0
3          12.0
4           1.0
           ... 
7316918     NaN
7316919     NaN
7316920    27.0
7316921    10.0
7316922     NaN
Name: runTimeMinutes, Length: 7316923, dtype: float64

In [10]:
batch_size = 10000
length = len(title_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_basics_preprocessed, batch_size),total=length):
    load_dataframe(chunk,'titlebasics')

 13%|██████████▌                                                                      | 96/732 [00:36<04:00,  2.65it/s]


DataError: (mysql.connector.errors.DataError) 1406 (22001): Data too long for column 'tconst' at row 9208
[SQL: INSERT INTO titlebasics (tconst, `titleType`, `primaryTitle`, `originalTitle`, `isAdult`, `startYear`, `endYear`, `runtimeMinutes`) VALUES (%(tconst)s, %(titleType)s, %(primaryTitle)s, %(originalTitle)s, %(isAdult)s, %(startYear)s, %(endYear)s, %(runtimeMinutes)s)]
[parameters: ({'tconst': 'tt0990629', 'titleType': 'tvEpisode', 'primaryTitle': 'Ancient Prophets: Could Ancients See the Future?', 'originalTitle': 'Ancient Prophets: Could Ancients See the Future?', 'isAdult': 0, 'startYear': datetime.datetime(2001, 1, 1, 0, 0), 'endYear': None, 'runtimeMinutes': 42.0}, {'tconst': 'tt0990630', 'titleType': 'tvEpisode', 'primaryTitle': "Bermuda Triangle: What's the Secret of the Bermuda Triangle?", 'originalTitle': "Bermuda Triangle: What's the Secret of the Bermuda Triangle?", 'isAdult': 0, 'startYear': datetime.datetime(2001, 1, 1, 0, 0), 'endYear': None, 'runtimeMinutes': None}

## Title ratings ##

In [37]:
# read data from title ratings file
title_ratings = pd.read_csv('title.ratings.tsv', delimiter='\t',encoding='utf-8')
title_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1658
1,tt0000002,6.1,201
2,tt0000003,6.5,1370
3,tt0000004,6.2,122
4,tt0000005,6.2,2158
...,...,...,...
1087762,tt9916580,7.2,5
1087763,tt9916690,6.6,5
1087764,tt9916720,6.0,66
1087765,tt9916766,6.9,14


In [38]:
def preprocess_dataframe_title_ratings(data_frame):
    title_ratings_new_series=data_frame
    
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'averageRating': 'float'})
    title_ratings_new_frame_avg_rating= title_ratings_new_series.astype({'numVotes': 'int32'})
  
    return title_ratings_new_frame_avg_rating
title_ratings_preprocessed=preprocess_dataframe_title_ratings(title_ratings)

In [39]:
len(title_ratings_preprocessed)

1087767

In [40]:
batch_size = 10000
length = len(title_ratings_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_ratings_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleratings')

100%|████████████████████████████████████████████████████████████████████████████████| 109/109 [00:17<00:00,  6.14it/s]


## Title Episodes ##

In [41]:
# read title episodes data
title_episodes = pd.read_csv('title.episode.tsv', delimiter='\t',encoding='utf-8')
title_episodes

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16
...,...,...,...,...
5273074,tt9916846,tt1289683,3,18
5273075,tt9916848,tt1289683,3,17
5273076,tt9916850,tt1289683,3,19
5273077,tt9916852,tt1289683,3,20


In [42]:
def preprocess_dataframe_title_episodes(data_frame):
    title_episodes_new_series=data_frame
    title_episodes_coerce_seasonNum = pd.to_numeric(title_episodes_new_series['seasonNumber'], errors='coerce')
    title_episodes_coerce_episodeNum = pd.to_numeric(title_episodes_new_series['episodeNumber'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_episodes_coerce_seasonNum)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_episodes_coerce_episodeNum)) 

    df_1['seasonNumber'] = df_1['test_column_1']
    df_2['episodeNumber'] = df_2['test_column_2']
    
    title_episodes_new_frame=title_episodes_new_series.drop(['seasonNumber'],axis=1)
    title_episodes_new_frame=title_episodes_new_series.drop(['episodeNumber'],axis=1)
    
    title_episodes_new_frame['seasonNumber']=df_1['seasonNumber']
    title_episodes_new_frame['episodeNumber']=df_2['episodeNumber']
       
    return title_episodes_new_frame

title_episodes_preprocessed = preprocess_dataframe_title_episodes(title_episodes)
title_episodes_preprocessed

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1.0,9.0
1,tt0042816,tt0989125,1.0,17.0
2,tt0042889,tt0989125,NaN,NaN
3,tt0043426,tt0040051,3.0,42.0
4,tt0043631,tt0989125,2.0,16.0
...,...,...,...,...
5273074,tt9916846,tt1289683,3.0,18.0
5273075,tt9916848,tt1289683,3.0,17.0
5273076,tt9916850,tt1289683,3.0,19.0
5273077,tt9916852,tt1289683,3.0,20.0


In [43]:
title_episodes_preprocessed.tconst = title_episodes_preprocessed.tconst.mask(~title_episodes_preprocessed.tconst.isin(title_basics_preprocessed.tconst),None)

In [44]:
batch_size = 10000
length = len(title_episodes_preprocessed)//batch_size+1

for chunk in tqdm(batch(title_episodes_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleepisodes')

100%|████████████████████████████████████████████████████████████████████████████████| 528/528 [01:58<00:00,  4.46it/s]


## Title Akas ##

In [45]:
title_akas = pd.read_csv('title.akas.tsv', delimiter='\t',encoding='utf-8')
title_akas

c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...
24001655,tt9916852,4,エピソード #3.20,JP,ja,\N,\N,0
24001656,tt9916852,5,Episódio #3.20,PT,pt,\N,\N,0
24001657,tt9916852,6,Episodio #3.20,IT,it,\N,\N,0
24001658,tt9916852,7,एपिसोड #3.20,IN,hi,\N,\N,0


In [46]:
def preprocess_dataframe_title_akas(data_frame):
    title_akas_new_series=data_frame
    title_akas_new_series=title_akas_new_series.drop(['types', 'attributes'],axis=1)
    
    title_akas_coerce_ordering = pd.to_numeric(title_akas_new_series['ordering'], errors='coerce')
    title_akas_coerce_isOriginalTitle = pd.to_numeric(title_akas_new_series['isOriginalTitle'], errors='coerce')

    df_1 = pd.DataFrame(dict(test_column_1 = title_akas_coerce_ordering)) 
    df_2 = pd.DataFrame(dict(test_column_2 = title_akas_coerce_isOriginalTitle)) 

    df_1['ordering'] = df_1['test_column_1']
    df_2['isOriginalTitle'] = df_2['test_column_2']
    
    title_akas_new_frame=title_akas_new_series.drop(['ordering'],axis=1)
    title_akas_new_frame=title_akas_new_series.drop(['isOriginalTitle'],axis=1)
    
    title_akas_new_frame['ordering']=df_1['ordering']
    title_akas_new_frame['isOriginalTitle']=df_2['isOriginalTitle']
    
    
    
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'title': 'float'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'language': 'float'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'region': 'float'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'ordering': 'int32'})
    #title_akas_new_frame_avg_rating= title_akas_new_series.astype({'isOriginalTitle': 'int32'})
    


    #print(title_ratings_new_frame_avg_rating)
    #title_ratings_new_frame_avg_rating.to_sql('titleratings',con=database_connection, if_exists='append',index=False)   
    return title_akas_new_frame
title_akas_preprocessed=preprocess_dataframe_title_akas(title_akas)

In [47]:
title_akas_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24001660 entries, 0 to 24001659
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   titleId          object 
 1   ordering         int64  
 2   title            object 
 3   region           object 
 4   language         object 
 5   isOriginalTitle  float64
dtypes: float64(1), int64(1), object(4)
memory usage: 1.1+ GB


In [48]:
title_akas_preprocessed[~title_akas_preprocessed.titleId.isin(title_basics_preprocessed.tconst)]

,titleId,ordering,title,region,language,isOriginalTitle
75604,tt0021006,1,"Ja, der Himmel über Wien",AT,\N,NaN
78167,tt0021453,1,Tapping Toes,US,\N,NaN
87302,tt0023019,1,Hollywood on Parade,US,\N,NaN
97708,tt0024677,1,Tom's in Town,US,\N,NaN
174326,tt0036165,1,Missing Men,US,\N,NaN
...,...,...,...,...,...,...
23960745,tt9894112,3,Épisode #2.5,FR,fr,0.0
23960746,tt9894112,4,Episodio #2.5,IT,it,0.0
23960747,tt9894112,5,エピソード #2.5,JP,ja,0.0
23960748,tt9894112,6,Episodio #2.5,ES,es,0.0


In [49]:
title_akas_preprocessed.titleId=title_akas_preprocessed.titleId.mask(~title_akas_preprocessed.titleId.isin(title_basics_preprocessed.tconst),None)

In [50]:
batch_size=10000
length=len(title_akas_preprocessed)//batch_size+1
for chunk in tqdm(batch(title_akas_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'titleAKAs')

  0%|                                                                                         | 0/2401 [00:00<?, ?it/s]c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\pandas\io\sql.py:1423: UserWarning: The provided table name 'titleAKAs' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
100%|██████████████████████████████████████████████████████████████████████████████| 2401/2401 [11:53<00:00,  3.36it/s]


## Name Basics

In [60]:
name_basics = pd.read_csv('name.basics.tsv', delimiter='\t',encoding='utf-8')
name_basics

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0031983,tt0072308,tt0050419,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0071877,tt0037382,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0059956,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0078723,tt0077975,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0050976,tt0083922,tt0050986"
...,...,...,...,...,...,...
10484335,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department",tt2455546
10484336,nm9993716,Essias Loberg,\N,\N,NaN,\N
10484337,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
10484338,nm9993718,Aayush Nair,\N,\N,cinematographer,\N


In [61]:
def preprocess_dataframe_name_basics(data_frame):
    name_basics_new_series=data_frame
    name_basics_new_series=name_basics_new_series.drop(['primaryProfession', 'knownForTitles'],axis=1)
    
    name_basics_new_series['birthYear']= pd.to_datetime(name_basics_new_series.birthYear, errors='coerce', format='%Y')

    #title_basics_new_frame_transpose['startYear']=[time.date() for time in title_basics_new_frame_transpose['startYear']]
    
    name_basics_new_series['deathYear']= pd.to_datetime(name_basics_new_series.deathYear, errors='coerce', format='%Y')
    
    return name_basics_new_series

name_basics_preprocessed = preprocess_dataframe_name_basics(name_basics)

name_basics_preprocessed["noofmovies"] = 0
name_basics_preprocessed["age"] = 0
name_basics_preprocessed["currentdate"] = 0

In [62]:
name_basics_preprocessed_new= pd.to_numeric(name_basics_preprocessed['noofmovies'], errors='coerce')
name_basics_preprocessed_new = pd.to_numeric(name_basics_preprocessed['age'], errors='coerce')
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed.currentdate, errors='coerce', format='%Y')


In [63]:
df_1 = pd.DataFrame(dict(test_column_1 = name_basics_preprocessed_new)) 
df_2 = pd.DataFrame(dict(test_column_2 = name_basics_preprocessed_new)) 

df_1['noofmovies'] = df_1['test_column_1']
df_2['age'] = df_2['test_column_2']

name_basics_preprocessed['noofmovies']=df_1['noofmovies']
name_basics_preprocessed['age']=df_2['age']
name_basics_preprocessed['currentdate']= pd.to_datetime(name_basics_preprocessed['currentdate'])

In [64]:
name_basics_preprocessed

,nconst,primaryName,birthYear,deathYear,noofmovies,age,currentdate
0,nm0000001,Fred Astaire,1899-01-01,1987-01-01,0,0,NaT
1,nm0000002,Lauren Bacall,1924-01-01,2014-01-01,0,0,NaT
2,nm0000003,Brigitte Bardot,1934-01-01,NaT,0,0,NaT
3,nm0000004,John Belushi,1949-01-01,1982-01-01,0,0,NaT
4,nm0000005,Ingmar Bergman,1918-01-01,2007-01-01,0,0,NaT
...,...,...,...,...,...,...,...
10484335,nm9993714,Romeo del Rosario,NaT,NaT,0,0,NaT
10484336,nm9993716,Essias Loberg,NaT,NaT,0,0,NaT
10484337,nm9993717,Harikrishnan Rajan,NaT,NaT,0,0,NaT
10484338,nm9993718,Aayush Nair,NaT,NaT,0,0,NaT


In [65]:
type(list(name_basics_preprocessed.columns[0]))

list

In [66]:
name_basics_preprocessed_new=name_basics_preprocessed.rename(columns={"nconst": "%snconst"})


In [67]:
name_basics_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10484340 entries, 0 to 10484339
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   nconst       object        
 1   primaryName  object        
 2   birthYear    datetime64[ns]
 3   deathYear    datetime64[ns]
 4   noofmovies   int64         
 5   age          int64         
 6   currentdate  datetime64[ns]
dtypes: datetime64[ns](3), int64(2), object(2)
memory usage: 559.9+ MB


In [68]:
batch_size = 10000
length = len(name_basics_preprocessed)//batch_size+1

for chunk in tqdm(batch(name_basics_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'namebasics')

100%|██████████████████████████████████████████████████████████████████████████████| 1049/1049 [04:57<00:00,  3.53it/s]


## Principals-will insert when nconst is in the right format  ##

In [69]:
title_principals = pd.read_csv('title.principals.tsv', delimiter='\t',encoding='utf-8')
title_principals

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
41795011,tt9916880,5,nm0996406,director,principal director,\N
41795012,tt9916880,6,nm1482639,writer,\N,\N
41795013,tt9916880,7,nm2586970,writer,books,\N
41795014,tt9916880,8,nm1594058,producer,producer,\N


In [70]:
title_principals_processed=title_principals.drop(['characters'],axis=1)

In [71]:
batch_size=10000
length=len(title_principals_processed)//batch_size+1

for chunk in tqdm(batch(title_principals_processed,batch_size),total=length):
    load_dataframe(chunk,'principals')

100%|██████████████████████████████████████████████████████████████████████████████| 4180/4180 [17:56<00:00,  3.88it/s]


## Title crew ##

In [74]:
title_crew = pd.read_csv('title.crew.tsv', delimiter='\t',encoding='utf-8')
title_crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
7316917,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7316918,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7316919,tt9916852,"nm5519375,nm5519454","nm6182221,nm1628284,nm2921377"
7316920,tt9916856,nm10538645,nm6951431


In [75]:
title_crew_processed=title_crew.drop(['directors'],axis=1)
title_crew_processed=title_crew.drop(['writers'],axis=1)

In [76]:
batch_size=10000
length=len(title_crew_processed)//batch_size+1

for chunk in tqdm(batch(title_crew_processed,batch_size),total=length):
    load_dataframe(chunk,'titlecrew')

  0%|▏                                                                                 | 2/732 [00:00<06:53,  1.77it/s]ERROR: Exception during reset or similar
Traceback (most recent call last):
  File "c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\sqlalchemy\pool\base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\sqlalchemy\pool\base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\sqlalchemy\dialects\mysql\base.py", line 2496, in do_rollback
    dbapi_connection.rollback()
  File "c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\mysql\connector\connection_cext.py", line 386, in rollback
    self._cmysql.rollback()
_mysql_connector.MySQLInterfaceError: MySQL server has gone away
100%|██████████████████████████████████████████████████████████████████████████

## Writers ##

In [77]:
def preprocess_dataframe_parsing(data_frame, column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(name=data_frame[column_name].str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    
    return data_frame
writers_preprocessed=preprocess_dataframe_parsing(title_crew.writers, "writers")

In [79]:
batch_size=1000
length=len(writers_preprocessed)//batch_size+1
for chunk in tqdm(batch(writers_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'writers')

  7%|█████                                                                         | 791/12102 [00:28<06:51, 27.46it/s]


MySQLInterfaceError: MySQL server has gone away

## Directors ##

In [80]:
directors_preprocessed=preprocess_dataframe_parsing(title_crew.directors, "directors")

ERROR: Exception during reset or similar
Traceback (most recent call last):
  File "c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\sqlalchemy\pool\base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\sqlalchemy\pool\base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\sqlalchemy\dialects\mysql\base.py", line 2496, in do_rollback
    dbapi_connection.rollback()
  File "c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\mysql\connector\connection_cext.py", line 386, in rollback
    self._cmysql.rollback()
_mysql_connector.MySQLInterfaceError: MySQL server has gone away


In [81]:
batch_size=10000
length=len(directors_preprocessed)//batch_size+1
for chunk in tqdm(batch(directors_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'directors')

100%|████████████████████████████████████████████████████████████████████████████████| 865/865 [02:42<00:00,  5.33it/s]


## Genres ##

In [ ]:
genres_preprocessed=preprocess_dataframe_parsing(title_basics.genres, "genres")

In [ ]:
batch_size=10000
length=len(genres_preprocessed)//batch_size+1
for chunk in tqdm(batch(genres_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'genres')

## Known for titles ##

In [82]:
knownForTitles_preprocessed=preprocess_dataframe_parsing(name_basics.knownForTitles, "knownForTitles")

In [83]:
batch_size=10000
length=len(knownForTitles_preprocessed)//batch_size+1
for chunk in tqdm(batch(knownForTitles_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'knownForTitles')

  0%|                                                                                         | 0/1834 [00:00<?, ?it/s]c:\users\rbren\appdata\local\programs\python\python38\lib\site-packages\pandas\io\sql.py:1423: UserWarning: The provided table name 'knownForTitles' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)
100%|██████████████████████████████████████████████████████████████████████████████| 1834/1834 [14:48<00:00,  2.06it/s]


## primary profession ##

In [ ]:
pprofession_preprocessed=preprocess_dataframe_parsing(name_basics.primaryProfession, "primaryProfession")

In [ ]:
batch_size=10000
length=len(pprofession_preprocessed)//batch_size+1
for chunk in tqdm(batch(pprofession_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'primaryProfession')

## types ## 

In [ ]:
types_preprocessed=preprocess_dataframe_parsing(title_akas.types, "types")

In [ ]:
batch_size=10000
length=len(types_preprocessed)//batch_size+1
for chunk in tqdm(batch(types_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'types')

## attributes ##

In [ ]:
attributes_preprocessed=preprocess_dataframe_parsing(title_akas.attributes, "attributes")

batch_size=100000
length=len(types_preprocessed)//batch_size+1
for chunk in tqdm(batch(types_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'attributes')

## Charcters ##

In [ ]:
def preprocess_dataframe_parsing(data_frame, column_name):
    
    data_frame=data_frame.to_frame()
    data_frame=data_frame.assign(=data_frame[column_name].str.replace('[\[\]"]',"").str.split(","))
    #type(x.iloc[0,1])
    data_frame=data_frame.explode('name')
    
    return data_frame
characters_preprocessed=preprocess_dataframe_parsing(title_principals.characters, "characters")

In [ ]:
batch_size=10000
length=len(characters_preprocessed)//batch_size+1
for chunk in tqdm(batch(directors_preprocessed,batch_size),total=length):
    load_dataframe(chunk,'characters')

## Question 5.  ##

In [104]:
conn = create_connection("localhost", "root", "Odyssey22", "imbd_application")
c = conn.cursor()

c.execute("DROP TABLE imbd_application")


Connection to MySQL DB successful


KeyboardInterrupt: 

In [106]:
conn = create_connection("localhost", "root", "Odyssey22", "imbd_application")
c = conn.cursor()

operation = """SELECT COUNT(KTN.knownfortitlesID)
               FROM namebasics AS NB, knownfortitlenames AS KTN
               WHERE KTN.knownfortitlesID IN (SELECT KT.knownfortitlesID 
                                              FROM knownforTitles AS KT 
                                              WHERE KT.titles IN (SELECT tconst
                                                                  FROM titlebasics AS TB 
                                                                  WHERE TB.titletype = 'Movie'))
                GROUP BY namebasicsID
                LIMIT 5"""

for result in c.execute(operation, multi=True):
    if result.with_rows:
        print("Rows produced by statement '{}':".format(
          result.statement))
        print(result.fetchall())
    else:
        print("Number of rows affected by statement '{}': {}".format(
          result.statement, result.rowcount))
        


The error '1049 (42000): Unknown database 'imbd_application'' occurred


AttributeError: 'NoneType' object has no attribute 'cursor'